In [1]:
#1 generate sentence with Transformer Model

import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.nn import Transformer
model = Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     output = model(src, tgt)
#     loss = criterion(output, tgt_labels)
#     loss.backward()
#     optimizer.step()

"""학습에 너무 오랜 시간이 걸리므로, 아래 명령어로 사전학습된 모델 호출"""
# pip install sentencepiece sacremoses

model = torch.hub.load('huggingface/pytorch-transformers', 'modelForCausalLM', 'gpt2')
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'gpt2')

input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids, max_length = 50, num_return_sequences = 1)
tokenizer.decode(output[0], skip_special_tokens=True)

Using cache found in C:\Users\sj5bl/.cache\torch\hub\huggingface_pytorch-transformers_main
Using cache found in C:\Users\sj5bl/.cache\torch\hub\huggingface_pytorch-transformers_main
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a'

In [4]:
#2 generate sentence with GPT 2

import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline

func = pipeline("text-generation", model = "gpt2")
result = func("I have a cat", max_length = 100, num_return_sequences = 2)
result

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'I have a cat because his mother died of leukemia, then I came home from work and we were seeing pictures of his body on a wall," he said. "We took a look at his body and when we looked at us his body would start moving. Our neighbor said \'how can this happen?\'"\n\nPolice and firefighters brought an 11-year-old girl to hospital, but her remains were pronounced dead from her injuries.\n\nA 17-year-old girl has been charged with'},
 {'generated_text': 'I have a cat I\'ve run away from, and she\'s still here. All kinds of things happen...But it\'s nice to know she\'s getting on with how she is. I have no fear of him being here anymore."\n\nI asked the boy what he was saying, "He\'s so beautiful. He\'s so quiet, he\'s very quiet. You just don\'t know if he can hear you." I asked how much I can expect him to get. (laughs)\n'}]

In [10]:
#3 sentiment analysis (감정어 분석)

import warnings
warnings.filterwarnings('ignore')

from transformers import pipeline

func = pipeline("sentiment-analysis")
result = func("I'm fine.")
result

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998200535774231}]

In [4]:
#4 gensim model을 활용한 단어간 유사도 계산

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from scipy.spatial.distance import cosine
import numpy as np

sentences = [
    "The quick brown fox jump over the lazy dog",
    "I love playing with my pet dog",
    "The dog barks at the stranger",
    "The cat sleeps on the sofa",
]

func = [simple_preprocess(i) for i in sentences]
print(func)

model = Word2Vec(sentences = func, vector_size=5, window =5, min_count=1, sg = 0)
# window : 단어를 예측할 때 고려할 주변 단어들의 범위를 지정. window=5 라면 현재 단어를 기준으로 좌우 5개의 단어를 문맥으로 고려하여 예측
# min_count : 전체 문장 리스트에서 등장 빈도가 해당값 미만인 단어는 학습에서 제외
# sg: 모델 유형 (0: CBOW, 1: Skip-gram, 기본 0)

dog = model.wv['dog']
cat = model.wv['cat']

# cosine 유사도 계산
sim = 1 - cosine(dog,cat)
print(sim)

[['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazy', 'dog'], ['love', 'playing', 'with', 'my', 'pet', 'dog'], ['the', 'dog', 'barks', 'at', 'the', 'stranger'], ['the', 'cat', 'sleeps', 'on', 'the', 'sofa']]
-0.16885965776110257


In [13]:
#5 BERT 기반으로 문장 간 유사도 판별

from transformers import BertModel, BertTokenizer
import torch
from scipy.spatial.distance import cosine

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

sentences = [
    "The quick brown fox jump over the lazy dog",
    "I love playing with my pet dog",
    "The dog barks at the stranger",
    "The cat sleeps on the sofa",
    "You don't love me.",
    "You hate me, actually.",
]

# return_tensors 종류
# 'pt': PyTorch 텐서로 반환. PyTorch를 사용하는 모델에 입력할 때 필요
# 'tf': TensorFlow 텐서로 반환. TensorFlow를 사용하는 모델에 입력할 때 필요
# 'np': NumPy 배열로 반환. 모델이나 연산에서 NumPy 배열을 사용할 때 필요
# None: 텐서나 배열 형식으로 변환하지 않고 그냥 파이썬 리스트 또는 딕셔너리로 반환

input1 = tokenizer(sentences[4], return_tensors='pt')
input2 = tokenizer(sentences[5], return_tensors='pt')

with torch.no_grad():
    output1 = model(**input1)
    output2 = model(**input2)

embedding1 = output1.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
embedding2 = output2.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# output.last_hidden_state.mean(dim=1): 모델의 마지막 히든 상태에서 각 문장의 단어 벡터를 평균하여 문장 벡터 반환
# squeeze(): 결과 텐서에서 불필요한 차원을 제거하여, 문장 임베딩이 (batch_size, hidden_size) 형태가 되도록 유도
# cpu(): 만약 GPU에서 계산된 텐서라면 이를 CPU로 옮김
# numpy(): 최종적으로 PyTorch 텐서를 Numpy 배열로 변환

# cosine 유사도 계산
sim = 1 - cosine(embedding1, embedding2)
print(sim)

0.8146521854087398


In [1]:
#6 언어 번역 모델 (성능 X, 참조용)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# NLLB-200 모델과 tokenizer 로드
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#번역할 문장
sentence = "사과 먹고 싶다."

#입력할 문장을 토큰화
encoded_sentence = tokenizer(sentence, return_tensors='pt')

# 번역 대상 언어 설정
tokenizer.src_lang = "Hangul"
# tokenizer.tgt_lang = "Latin"  # 영어로 번역

#번역 수행
func = model.generate(encoded_sentence.input_ids, forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"), max_length=30)

#번역 결과 디코딩
result = tokenizer.decode(func[0], skip_special_tokens=True)
result

'I want to apologize.'